In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
train_url = '/kaggle/input/correction-dset/train_corr.csv'
test_url = '/kaggle/input/correction-dset/test_corr.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [3]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [3]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [4]:
df_test

,Unnamed: 0,Video ID,Channel name,Time of Publishing,Title,Genre,Comment,Error,Category,Correct form
0,0,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,এই সব গান শুনলে মন ভরে যায়,0,NaN,এই সব গান শুনলে মন ভরে যায়
1,1,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,মনটা ভরে গেলো ভাই।অনেক সুন্দর হয়েছে,0,NaN,মনটা ভরে গেলো ভাই।অনেক সুন্দর হয়েছে
2,2,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,বাংলা সেরা গানটা আবার শুনছি..,0,NaN,বাংলা সেরা গানটা আবার শুনছি..
3,3,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,সুমন রায় এর কন্ঠে মে কোন গান ই আমার অনেক ভালো...,0,NaN,সুমন রায় এর কন্ঠে মে কোন গান ই আমার অনেক ভালো...
4,4,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,রতীন্দ্রনাত দাদুর কন্ঠে সবচেয়ে বেশি ভাল লাগে এ...,0,NaN,রতীন্দ্রনাত দাদুর কন্ঠে সবচেয়ে বেশি ভাল লাগে এ...
...,...,...,...,...,...,...,...,...,...,...
5017,5017,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,নাসির হোসেন বর্তমানে বাংলাদেশের একটা ভালো অলরা...,1,Spelling,নাসির হোসেন বর্তমানে বাংলাদেশের একটা ভালো অলরা...
5018,5018,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,তামিম ইকবাল কোথায় ওনার না এশিয়া কাপ থেকে খেল...,1,Spelling,তামিম ইকবাল কোথায় উনার না এশিয়া কাপ থেকে খেল...
5019,5019,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,লিটন এন্ড তাওহীদ রিদয় কই কেমন টিম হলো এটা ����,1,Code Switching,লিটন এন্ড তাওহীদ রিদয় কই কেমন দল হলো এটা ����
5020,5020,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,"সাব্বির একটা ম্যাচ ও ভালো খেলে না,,,কেন নে ওরে...",1,Spelling,"সাব্বির একটা ম্যাচ ও ভালো খেলে না,,,কেন নেয় ওক..."


In [5]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Error)
Test_Y = Encoder.fit_transform(df_test.Error)

In [6]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [8]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5219    0.3063    0.3860      1910
           1     0.6603    0.8278    0.7346      3112

    accuracy                         0.6294      5022
   macro avg     0.5911    0.5670    0.5603      5022
weighted avg     0.6077    0.6294    0.6020      5022



In [8]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {'C': [0.1, 1, 10, 100],
#               'kernel': ['linear', 'rbf'],
#               'degree': [2, 3, 4],
#               'gamma': ['auto', 'scale']}

# grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=3)
# grid.fit(Train_X_Tfidf, Train_Y)

# print("Best Parameters:", grid.best_params_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END C=0.1, degree=2, gamma=auto, kernel=linear;, score=0.620 total time=  20.6s
[CV 2/5] END C=0.1, degree=2, gamma=auto, kernel=linear;, score=0.621 total time=  20.7s
[CV 3/5] END C=0.1, degree=2, gamma=auto, kernel=linear;, score=0.625 total time=  20.4s
[CV 4/5] END C=0.1, degree=2, gamma=auto, kernel=linear;, score=0.623 total time=  20.2s
[CV 5/5] END C=0.1, degree=2, gamma=auto, kernel=linear;, score=0.624 total time=  20.4s
[CV 1/5] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=0.620 total time=  21.3s
[CV 2/5] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=0.620 total time=  21.2s
[CV 3/5] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=0.620 total time=  20.7s
[CV 4/5] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=0.620 total time=  20.7s
[CV 5/5] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=0.620 total time=  21.0s
[CV 1/5] END C=0.1, degree=2, gamma=scale, kernel=linear;, scor

In [12]:
SVM_predictions = SVM.predict(Test_X_Tfidf)
# param_grid = {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
SVM = svm.SVC(C=1, kernel='rbf', degree=2, gamma='scale')
SVM.fit(Train_X_Tfidf, Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print(metrics.classification_report(Test_Y, predictions_SVM, digits=4))


              precision    recall  f1-score   support

           0     0.5489    0.2969    0.3853      1910
           1     0.6633    0.8503    0.7452      3112

    accuracy                         0.6398      5022
   macro avg     0.6061    0.5736    0.5653      5022
weighted avg     0.6198    0.6398    0.6084      5022

